In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
stats = pd.read_csv('test1.txt', 
                    names=['E', 'S', 'T', 'a', 'b', 'x', 'N', 'p', 'P'],
                    sep = " ")
stats['T'] = list(map((lambda x: float(x[:-1])), stats['T']))
stats.head()

In [ ]:
data = []
for i in [1, 2, 4, 8, 16]:
    data.append((i, stats[stats['P'] == i]))

In [ ]:
plt.figure(figsize=(15, 10))

for data_i in data:
    plt.plot(data_i[1]['N'], np.array(data_i[1]['T']), alpha=0.3, label=None)
    plt.scatter(data_i[1]['N'], np.array(data_i[1]['T']), label='P = '+str(data_i[0]))

plt.title('T(N)', y=1.03, fontsize=22)
plt.xlabel('N', fontsize=16)
plt.ylabel('T', fontsize=16)

plt.legend(fontsize=12)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

for data_i in data:
    plt.plot(data_i[1]['N'], np.array(data_i[1]['S']), alpha=0.3, label=None)
    plt.scatter(data_i[1]['N'], np.array(data_i[1]['S']), label='S = '+str(data_i[0]))

plt.title('S(N)', y=1.03, fontsize=22)
plt.xlabel('N', fontsize=16)
plt.ylabel('S', fontsize=16)

plt.legend(fontsize=12)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

for data_i in data:
    plt.plot(data_i[1]['N'], np.array(data_i[1]['E']), alpha=0.3, label=None)
    plt.scatter(data_i[1]['N'], np.array(data_i[1]['E']), label='E = '+str(data_i[0]))

plt.title('E(N)', y=1.03, fontsize=22)
plt.xlabel('N', fontsize=16)
plt.ylabel('E', fontsize=16)

plt.legend(fontsize=12)
plt.grid()
plt.show()

In [ ]:
stats = pd.read_csv('test2.txt', 
                    names=['E', 'S', 'T', 'a', 'b', 'x', 'N', 'p', 'P'],
                    sep = " ")
stats['T'] = list(map((lambda x: float(x[:-1])), stats['T']))
stats.head()

In [ ]:
plt.figure(figsize=(15, 10))

plt.plot(stats['P'], np.array(stats['T']), alpha=0.3, label=None)
plt.scatter(stats['P'], np.array(stats['T']), label='plot')

plt.title('T(P)', y=1.03, fontsize=22)
plt.xlabel('N', fontsize=16)
plt.ylabel('T', fontsize=16)

plt.legend(fontsize=12)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

plt.plot(stats['P'], np.array(stats['S']), alpha=0.3, label=None)
plt.scatter(stats['P'], np.array(stats['S']), label='plot')

plt.title('S(P)', y=1.03, fontsize=22)
plt.xlabel('N', fontsize=16)
plt.ylabel('S', fontsize=16)

plt.legend(fontsize=12)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

plt.plot(stats['P'], np.array(stats['E']), alpha=0.3, label=None)
plt.scatter(stats['P'], np.array(stats['E']), label='plot')

plt.title('E(P)', y=1.03, fontsize=22)
plt.xlabel('N', fontsize=16)
plt.ylabel('E', fontsize=16)

plt.legend(fontsize=12)
plt.grid()
plt.show()

In [ ]:
import collections
import collections.abc

from enum import Enum

import threading, time, random 

import datetime
import calendar

from telegram.ext import Filters
from telegram.ext import MessageHandler
from telegram import ReplyKeyboardMarkup
from telegram import ReplyMarkup
from telegram.ext import Updater

class Message(object):
    def __init__(self, text, **options):
        self.text = text
        self.options = options


class Markdown(Message):
    def __init__(self, text, **options):
        super(Markup, self).__init__(text, parse_mode="Markdown", **options)

    def __repr__(self):
        options = dict(self.options)
        options.pop("parse_mode")
        options = (", " + repr(options)) if options else ""
        return "Markdown(%r%s)" % (self.text, options)


class HTML(Message):
    def __init__(self, text, **options):
        super(HTML, self).__init__(text, parse_mode="HTML", **options)

    def __repr__(self):
        options = dict(self.options)
        options.pop("parse_mode")
        options = (", " + repr(options)) if options else ""
        return "HTML(%r%s)" % (self.text, options)

class DialogBot(object):
    def __init__(self, token, generator, handlers=None):
        self.updater = Updater(token=token)
        handler = MessageHandler(Filters.text | Filters.command, self.handle_message)
        self.updater.dispatcher.add_handler(handler)
        self.handlers = collections.defaultdict(generator, handlers or {})

    def start(self):
        self.updater.start_polling()
        self.updater.idle()

    def handle_message(self, bot, update):
        print("New message");
        chat_id = update.message.chat_id
        if update.message.text == "/start":
            self.handlers.pop(chat_id, None)
        
        if chat_id not in self.handlers:
            answer = next(self.handlers[chat_id])
        else:
            try:
                answer = self.handlers[chat_id].send(update.message)
            except StopIteration:
                del self.handlers[chat_id]
                return self.handle_message(bot, update)
        self._send_answer(bot, chat_id, answer)

    def _send_answer(self, bot, chat_id, answer):
        print("Sending answer %r to %s" % (answer, chat_id))
        if isinstance(answer, collections.abc.Iterable) and not isinstance(answer, str):
            # мы получили несколько объектов -- сперва каждый надо обработать
            answer = list(map(self._convert_answer_part, answer))
        else:
            # мы получили один объект -- сводим к более общей задаче
            answer = [self._convert_answer_part(answer)]
        # перед тем, как отправить очередное сообщение, идём вперёд в поисках
        # «довесков» -- клавиатуры там или в перспективе ещё чего-нибудь
        current_message = None
        for part in answer:
            if isinstance(part, Message):
                if current_message is not None:
                    # сообщение, которое мы встретили раньше, пора бы отправить.
                    options = dict(current_message.options)
                    # поскольку не все объекты исчерпаны, пусть это сообщение
                    # не вызывает звоночек (если не указано обратное)
                    options.setdefault("disable_notification", True)
                    bot.sendMessage(chat_id=chat_id, text=current_message.text, **options)
                current_message = part
                current_message.options["reply_markup"] = None
            if isinstance(part, ReplyMarkup):
                # ага, а вот и довесок! добавляем текущему сообщению.
                # нет сообщения -- ну извините, это ошибка.
                current_message.options["reply_markup"] = part
        # надо не забыть отправить последнее встреченное сообщение.
        if current_message is not None:
            bot.sendMessage(chat_id=chat_id, text=current_message.text, **current_message.options)

    def _convert_answer_part(self, answer_part):
        if isinstance(answer_part, str):
            return Message(answer_part)
        if isinstance(answer_part, collections.abc.Iterable):
            # клавиатура?
            answer_part = list(answer_part)
            if isinstance(answer_part[0], str):
                # она! оформляем как горизонтальный ряд кнопок.
                # кстати, все наши клавиатуры одноразовые -- нам пока хватит.
                return ReplyKeyboardMarkup([answer_part], resize_keyboard=True, one_time_keyboard=False)
            elif isinstance(answer_part[0], collections.abc.Iterable):
                # двумерная клавиатура?
                answer_part = list(map(list, answer_part))
                if isinstance(answer_part[0][0], str):
                    # она!
                    return ReplyKeyboardMarkup(answer_part, resize_keyboard=True, one_time_keyboard=False)
        return answer_part

class Task(Enum):
    NONE = 0
    CREATE_EVENT = 1
    SHOW_TIMETABLE = 2
    
def dialog():
    task = yield from ask_about_task()
    print(task)
    if   task == Task.CREATE_EVENT:
        yield from discuss_create_event()
    elif task == Task.SHOW_TIMETABLE:
        yield from discuss_show_timetable()

def ask_about_task():
    task = Task.NONE
    answer = yield (HTML('<b>How can I help you?</b>\n<b>1. </b>Create event\n<b>2. </b>Show timetable'), 
                        [['📝 Create event '], ['📆 Show timetable']])
    while task == Task.NONE:
        if 'create' in answer.text.lower():
            task = Task.CREATE_EVENT
        elif 'show' in answer.text.lower():
            task = Task.SHOW_TIMETABLE
        else:
            answer = yield (HTML('Missed that 😒\n\n<b>How can I help you?</b>\n<b>1. </b>Create event\n<b>2. </b>Show timetable'), 
                        [['📝 Create event '], ['📆 Show timetable']])
            
    return task

def discuss_create_event():
    answer = yield ("Give a name of your event.")
    name = answer.text
    date = answer.date
    start_date = yield from ask_date("When it starts?", date)
    end_data   = yield from ask_date("And when ends?", date)
    yield str(name) + "\n" + str(start_date) + "\n" + str(end_date)
    
def discuss_show_timetable():
    answer = yield ("Not supported yet too 🙄", [['⬅️ Back']])
    while not ('back' in answer.text.lower()):
        answer = yield ("Missed that 😒", [['⬅️ Back']])

def get_date_keyboard(date):
    keyboard = []
    line = []
#     keyboard.append(['⬅️ Back'])
    line.append([])
    line.append('⬅️')
    line.append('<')
    line.append(str(date.strftime("%b")))
    line.append(str(date.year))
    line.append('>')
    keyboard.append(line)
    line = []
    
    first_mounth_day = datetime.datetime(date.year, date.mounth, 1)
    num_days = calendar.monthrange(date.year, date.month)[1]

    pos = 0
    for i in range(first_weekday):
        line.append(' ')
        pos += 1
    for day in range(num_days):
        line.append(str(day + 1))
        pos += 1
        if pos % 7 == 0:
            keyboard.append(line)
            line = []
    if pos % 7 != 0
        for i in range(7 - pos % 7):
            line.append(' ')
        keyboard.append(line)
        line = []
    return keyboard
    
        
def ask_date(question, date):
    while (True):
        keyboard = get_date_keyboard(date)
        answer = yield (question, keyboard)
        if answer.text == '<':
            if date.month == 1:
                date = datetime.datetime(date.year - 1, 12, 1)
            else:
                date = datetime.datetime(date.year, date.month - 1, 1)
        elif answer.text == '>':
            if date.month == 12:
                date = datetime.datetime(date.year + 1, 1, 1)
            else:
                date = datetime.datetime(date.year, date.month + 1, 1)
        else:
            break
    return 0


TOKEN = "372828093:AAFIgqIHvneYjO-4PIrN8RF3mP9fzkB88-Q"

if __name__ == "__main__":
    global TOKEN
    dialog_bot = DialogBot(TOKEN, dialog)
    dialog_bot.start()

In [ ]:
from datetime import datetime

str(datetime.fromtimestamp(10))

In [ ]:
t = datetime.date.today()
get_date_keyboard(t)

In [2]:
# -*- coding: utf-8 -*-

from telegram.ext import Filters
from telegram.ext import MessageHandler
from telegram import ReplyKeyboardMarkup
from telegram import ReplyMarkup
from telegram.ext import Updater

bot = telebot.TeleBot("372828093:AAFIgqIHvneYjO-4PIrN8RF3mP9fzkB88-Q")


@bot.message_handler(content_types=["text"])
def any_msg(message):
    # Создаем клавиатуру и каждую из кнопок (по 2 в ряд)
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    url_button = types.InlineKeyboardButton(text="URL", url="https://ya.ru")
    callback_button = types.InlineKeyboardButton(text="Callback", callback_data="test")
    switch_button = types.InlineKeyboardButton(text="Switch", switch_inline_query="Telegram")
    keyboard.add(url_button, callback_button, switch_button)
    bot.send_message(message.chat.id, "Я – сообщение из обычного режима", reply_markup=keyboard)


@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    # Если сообщение из чата с ботом
    if call.message:
        if call.data == "test":
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="Пыщь")
            # Уведомление в верхней части экрана
            bot.answer_callback_query(callback_query_id=call.id, show_alert=False, text="Пыщь!")
    # Если сообщение из инлайн-режима
    elif call.inline_message_id:
        if call.data == "test":
            bot.edit_message_text(inline_message_id=call.inline_message_id, text="Бдыщь")


# Простейший инлайн-хэндлер для ненулевых запросов
@bot.inline_handler(lambda query: len(query.query) > 0)
def query_text(query):
    kb = types.InlineKeyboardMarkup()
    kb.add(types.InlineKeyboardButton(text="Нажми меня", callback_data="test"))
    results = []
    # Обратите внимание: вместо текста - объект input_message_content c текстом!
    single_msg = types.InlineQueryResultArticle(
        id="1", title="Press me",
        input_message_content=types.InputTextMessageContent(message_text="Я – сообщение из инлайн-режима"),
        reply_markup=kb
    )
    results.append(single_msg)
    bot.answer_inline_query(query.id, results)


if __name__ == '__main__':
    bot.polling(none_stop=True)

NameError: name 'telebot' is not defined